In [1]:
import pandas as pd
import numpy as np

In [ ]:
pieces = []
years = range(1880, 2011)
columns = ['name', 'sex', 'births']

for year in years:
    path = 'Names/yob%d.txt' % year
    frame = pd.read_csv(path, names = columns)
    frame['year'] = year
    pieces.append(frame)
    names = pd.concat(pieces, ignore_index=True)
    
print(names)

names.to_pickle('names.pkl')

In [3]:
names = pd.read_pickle('names.pkl')

In [31]:
def get_topX(group, X):
    return group.sort_values(by='births', ascending = False)[:X]


def get_topK(year, k):
    grouped = names.groupby(['year'])
    topk = grouped.apply(get_topX, k)
    
    ret_ind = np.where(topk['year'] == year)[0].tolist()

    return(topk.iloc[ret_ind])

def frequency(name, year):
    ret_ind1 = np.where(names['year'] == year)[0].tolist()
    tdf = names.iloc[ret_ind1]
    
    ret_ind2 = np.where(tdf['name'] == name)[0].tolist()
    ndf = tdf.iloc[ret_ind2]
    
    return ndf['births'].sum()

def relative_frequency(name, year):
    ret_ind1 = np.where(names['year'] == year)[0].tolist()
    tdf = names.iloc[ret_ind1]
    
    ret_ind2 = np.where(tdf['name'] == name)[0].tolist()
    ndf = tdf.iloc[ret_ind2]
    
    return ndf['births'].sum() / tdf['births'].sum()

def freq_G(name, gender, year):
    ret_ind1 = np.where(names['year'] == year)[0].tolist()
    tdf = names.iloc[ret_ind1]
    
    ret_ind2 = np.where(tdf['name'] == name)[0].tolist()
    ndf = tdf.iloc[ret_ind2]
    
    ret_ind3 = np.where(ndf['sex'] == gender)[0].tolist()
    sdf = ndf.iloc[ret_ind3]
    
    return sdf['births'].sum()
    
    
print(get_topK(1930, 3))

print(frequency('Adam', 1976))

print(relative_frequency('Adam', 1976))

def get_swaps():
    a = np.empty(0)
    
    name2 = names['name']
    name2.drop_duplicates(inplace=True)
    n = name2.sort_values(ascending=True)
    
    for name in n:
        if(did_swap(name)):
            np.append(a, name)
            
    return a        
        
        
def did_swap(name):
    fmp = False
    mmp = False
    
    m1880 = freq_G(name, 'M', 1880)
    m2011 = freq_G(name, 'M', 2010)
    f1880 = freq_G(name, 'F', 1880)
    f2011 = freq_G(name, 'F', 2010)
    
    if(m1880 == 0 and f1880 == 0):
        return False
    if(m2011 == 0 and f2011 == 0):
        return False
    
    mmp1880 = False
    mmp2011 = False
    
    if(m1880 > f1880):
        mmp1880 = True
    if(m2011 > f2011):
        mmp2011 = True

    if(mmp1880 != mmp2011):
        return True
    else:
        return False
          
            
#print(did_swap('Ash'))

a = get_swaps()
print('Gender Swaps:')
print(a)

               name sex  births  year
year                                 
1930 274818    Mary   F   64137  1930
     280067  Robert   M   62146  1930
     280068   James   M   53946  1930
9991
0.0032917659067149586
Aaron
102 7458 0 23
Ab
5 6 0 0
Abbie
0 0 71 327
Abbott
5 34 0 0
Abby
0 0 6 1147
Abe
50 54 0 0
Abel
9 1125 0 0
Abigail
0 9 12 14231
Abner
27 162 0 0
Abraham
81 1923 0 0
Abram
21 488 0 0
Ada
0 7 652 535
Adah
0 0 24 76
Adaline
0 0 23 135
Adam
104 5096 0 9
Adda
0 0 14 5
Addie
8 0 274 206
Addison
19 192 0 10320
Swapped: Addison
Adela
0 0 9 128
Adelaide
0 0 65 711
Adele
0 0 41 287
Adelia
0 0 18 51
Adeline
0 0 54 989
Adella
0 0 26 65
Adelle
0 0 5 147
Aden
7 1171 0 17
Adina
0 0 5 114
Adolph
93 7 0 0
Adrian
18 7397 0 163
Adrienne
0 0 5 409
Agatha
0 0 16 49
Agnes
0 0 473 67
Agustus
5 10 0 0
Aileen
0 0 6 645
Aimee
0 0 13 474
Al
12 20 0 0
Alba
0 0 6 115
Albert
1493 697 0 0
Alberta
0 0 76 16
Albin
6 15 0 0
Albina
0 0 7 19
Albion
6 13 0 0
Alda
0 0 6 6
Alden
9 251 0 25
Alec
11 665 0 5
Al

Elnora
0 0 42 10
Eloise
0 0 11 552
Elroy
5 12 0 0
Elsa
0 0 20 487
Elsie
6 0 301 457
Elton
7 56 0 0
Elva
0 0 44 54
Elvin
10 137 0 0
Elvina
0 0 5 6
Elvira
0 0 30 41
Elwood
23 19 0 0
Ely
5 137 0 21
Elza
25 0 0 10
Swapped: Elza
Elzie
13 6 0 0
Emanuel
57 1191 0 0
Emelia
0 0 12 338
Emelie
0 0 10 61
Emeline
0 0 9 76
Emerson
25 635 0 1009
Swapped: Emerson
Emery
52 311 0 1023
Swapped: Emery
Emil
119 106 0 0
Emile
12 32 0 0
Emilia
0 0 9 923
Emilie
0 0 46 380
Emily
0 20 210 14262
Emma
10 17 2003 17327
Emmaline
0 0 14 204
Emmet
34 131 0 0
Emmett
74 1276 0 0
Emmie
0 0 12 177
Emmitt
7 241 0 0
Emory
40 184 0 230
Swapped: Emory
Ena
0 0 5 29
Enoch
60 196 0 0
Enos
14 29 0 0
Enrique
5 878 0 0
Ephraim
18 180 0 0
Era
0 0 11 15
Erasmus
5 5 0 0
Eric
10 4192 0 7
Erie
6 0 6 5
Erma
0 0 30 13
Erna
0 0 10 9
Ernest
615 218 0 0
Ernie
6 27 0 0
Ervin
30 107 0 0
Erwin
16 91 0 0
Esau
5 57 0 0
Essie
0 0 78 12
Esta
0 0 9 5
Estella
0 0 162 250
Estelle
0 0 119 157
Esther
0 6 198 1253
Ethel
6 0 633 14
Etta
0 0 323 81
Eudora

Linwood
5 8 0 0
Lionel
8 228 0 0
Lissie
0 0 11 5
Liza
0 0 18 93
Lizzie
0 0 388 47
Llewellyn
9 11 0 0
Lloyd
104 127 0 0
Logan
13 14014 0 620
Lois
0 0 53 69
Lola
0 0 138 1473
Lona
0 0 28 19
Lonnie
53 125 13 6
Lonzo
10 9 0 0
Lora
0 0 74 52
Loran
5 6 0 7
Swapped: Loran
Loren
22 70 0 109
Swapped: Loren
Lorena
0 0 29 263
Lorene
0 0 5 21
Lorenzo
48 964 0 0
Loretta
0 0 41 96
Lorin
5 8 0 15
Swapped: Lorin
Lottie
0 0 245 32
Lou
14 16 131 11
Swapped: Lou
Louella
0 0 34 21
Louie
33 125 14 0
Louis
828 916 5 0
Louisa
0 0 126 162
Louise
0 0 635 150
Love
0 12 5 90
Lovie
0 0 14 9
Lovina
0 0 8 13
Loyd
14 9 0 0
Lucia
0 0 15 1237
Lucian
18 447 0 0
Lucie
0 0 23 145
Lucien
19 169 0 0
Lucile
0 0 40 22
Lucille
0 0 40 630
Lucina
0 0 5 17
Lucinda
0 0 63 190
Lucious
10 24 0 0
Lucius
38 128 0 0
Lucretia
0 0 29 8
Lucy
0 0 590 3633
Ludwig
9 5 0 0
Luella
0 0 111 102
Luis
12 5126 0 6
Luke
43 8835 0 6
Lula
0 0 621 67
Lulu
0 0 249 51
Luna
0 7 15 940
Lura
0 0 53 9
Luther
260 73 0 0
Lyda
0 0 32 5
Lydia
0 6 302 2830
Lyle


Susie
0 0 361 82
Suzanne
0 0 6 88
Sybilla
0 0 6 8
Sydney
12 73 0 4328
Swapped: Sydney
Sylvester
89 62 0 0
Sylvia
0 0 81 525
Tabitha
0 0 8 463
Taylor
37 955 0 5892
Swapped: Taylor
Ted
13 81 0 0
Tena
0 0 19 9
Tennessee
0 12 5 10
Swapped: Tennessee
Teresa
0 0 50 409
Teressa
0 0 5 10
Terry
10 520 0 21
Tessie
0 0 17 15
Texas
0 9 5 6
Swapped: Texas
Thad
6 24 0 0
Thaddeus
12 198 0 0
Thea
0 0 6 143
Thelma
0 0 5 20
Theo
9 221 7 0
Theodora
0 0 11 47
Theodore
232 1330 0 0
Theophile
5 8 0 0
Theresa
0 0 153 280
Therese
0 0 11 70
Theron
7 96 0 0
Thomas
2534 7115 8 11
Thurman
8 13 0 0
Tilda
0 0 7 9
Tilden
8 12 0 0
Tillie
0 0 83 23
Tillman
9 28 0 0
Tim
12 65 0 0
Timothy
70 3441 0 5
Tina
0 0 17 161
Titus
6 712 0 0
Tobias
7 495 0 0
Tom
399 80 0 0
Tomas
18 384 0 0
Tommie
15 58 10 11
Tommy
5 420 0 0
Toney
7 20 0 0
Tony
42 746 0 5
Tracy
6 77 0 126
Swapped: Tracy
Travis
12 2120 0 0
Tressa
0 0 7 26
Tressie
0 0 6 10
Troy
8 1382 0 8
Truman
12 197 0 0
Turner
11 174 0 11
Tyler
7 10436 0 197
Ula
0 0 5 8
Ulysses
2